In [11]:
from datetime import datetime as dt
import re
import pandas as pd
from xbbg import blp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from datetime import timedelta as td
import statsmodels.api as sm
import re
import matplotlib.image as image
from ipywidgets import interact, IntSlider, Checkbox, Dropdown, Output, HBox, VBox, interactive, interactive_output, ToggleButton,Text, Button, DatePicker, IntText, ToggleButtons, RadioButtons,SelectMultiple
from IPython.display import display, clear_output
import itertools
from scipy import stats
from scipy.optimize import minimize 
from scipy.special import ndtr


# Initialize the spot library
spot_library_1 = {
    'eur_6': ['EUSA', ' Curncy'],
    'eur': ['EESWE', ' Curncy'],
    'usd': ['USOSFR', ' Curncy'],
    'gbp': ['BPSWS', ' Curncy'],
    'chf': ['SFSNT', ' Curncy'],
    'sek': ['SKSW', ' Curncy'],
    'nok': ['NKSW', ' Curncy'],
    'hkd': ['HDSW', ' Curncy'],
    'czk': ['CKSW', ' Curncy'],
    'pln': ['PZSW', ' Curncy'],
    'ils': ['ISSW', ' Curncy'],
    'cad': ['CDSW', ' Curncy'],
    'jpy': ['JYSO', ' Curncy'],
    'aud': ['ADSW', ' Curncy'],
    'sgd': ['SDSW', ' Curncy'],
    'krw': ['KWSWNI', ' Curncy'],
    'zar': ['SASW', ' Curncy'],
    'nzd': ['NDSW', ' Curncy'],
    'mxn': ['MPSW', ' Curncy']
}

spot_library = {'eur_6':['EUSA', ' Curncy'],'eur': ['EESWE', ' Curncy'], 'usd': ['USOSFR', ' Curncy'], 
                'gbp': ['BPSWS', ' Curncy'],'chf': ['SFSNT', ' Curncy'],'sek': ['SKSW', ' Curncy'],
                 'nok': ['NKSW', ' Curncy'], 'hkd': ['HDSW', ' Curncy'],'czk': ['CKSW', ' Curncy'],
                 'pln': ['PZSW', ' Curncy'],'ils':['ISSW', ' Curncy'],  'cad':['CDSW', ' Curncy'], 
                 'jpy':['JYSO', ' Curncy'], 'aud': ['ADSW', ' Curncy'],'sgd':['SDSW', ' Curncy'],
                'krw': ['KWSWNI', ' Curncy'],
                'zar': ['SASW', ' Curncy'],
                'nzd': ['NDSW', ' Curncy'],
                'mxn': ['MPSW', ' Curncy']} 

forward_library = {'eur_6': ['EUSA', ' Curncy'], 
                 'eur': ['S0514FS ', ' BLC Curncy'], 
                 'usd': ['S0490FS ', ' BLC Curncy'], 
                 'gbp': ['S0141FS ', ' BLC Curncy'],
                 'chf': ['S0234FS ', ' BLC Curncy'],
                 'sek': ['SD0020FS ', ' BLC Curncy'],
                 'nok': ['SD0313FS ', ' BLC Curncy'],
                 'hkd': ['HDFS', ' Curncy'],
                 'czk': ['S0320FS ', ' BLC Curncy'],
                 'pln': ['S0323FS ', ' BLC Curncy'],
                 'ils': ['ISFS', ' Curncy'],
                 'cad': ['CDFS', ' Curncy'],
                 'jpy': ['S0195FS ', ' BLC Curncy'],
                 'aud': ['SD0302FS ', ' BLC Curncy'],
                'sgd': ['SDFS', ' Curncy'],
                'krw': ['S0205FS ', ' BLC Curncy'],
                'zar': ['SAFS', ' Curncy'],
                'nzd': ['SD0015FS ', ' BLC Curncy'],
                'mxn': ['SD0083FS ', ' BLC Curncy']} 

def spot_ticker(dex,tenor):
    if dex.lower() == 'mxn':
        y = int(tenor[:-1]) * 13
        num = f'{y//12}{chr(64+(y%12))}'
    else:
        num = tenor[:-1]
    return f'{spot_library[dex][0]}{num}{spot_library[dex][1]}'

def forward_ticker(dex,fwd):
    dex = dex.lower()
    fwd = fwd.lower()
    old = ['eur_6','hkd','ils','cad','sgd','zar']
    
    if cut(fwd)[0] == '0y':
        return spot_ticker(dex,cut(fwd)[1])
    elif dex == 'eur_6':
        F,T = f(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    elif dex in old:
        F,T = t(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    else:
        
        return f'{forward_library[dex][0]}{fwd.upper()}{forward_library[dex][1]}'

# Swap structure builders
def cut(some):
    x = 0
    y = some[0]
    while y.isdigit():
        y = some[x]
        x += 1
    return some[:x], some[x:]

def f(tenor):
    x = re.findall(r'\d+', tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 > 0:
            num += '1'
        num += chr(64 + (int(x) % 12))
    else:
        if len(x) == 1:
            num += '0'
        num += x
    return num

def t(tenor):
    x = re.findall(r'\d+', tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 > 0:
            num += '1'
        else:
            num += '0'
        num += chr(64 + (int(x) % 12))
    else:
        if len(x) == 1:
            num += '0'
        num += x
    return num

def spot_ticker1(dex, tenor):
    if dex.lower() == 'mxn':
        y = int(tenor[:-1]) * 13
        num = f'{y // 12}{chr(64 + (y % 12))}'
    else:
        num = tenor[:-1]
    return f'{spot_library_1[dex][0]}{num}{spot_library_1[dex][1]}'

def forward_ticker1(dex, fwd):
    dex = dex.lower()
    fwd = fwd.lower()
    old = ['eur_6', 'hkd', 'ils', 'cad', 'sgd', 'zar']

    if cut(fwd)[0] == '0y':
        return spot_ticker(dex, cut(fwd)[1])
    elif dex == 'eur_6':
        F, T = f(cut(fwd)[0]), t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    elif dex in old:
        F, T = t(cut(fwd)[0]), t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    else:
        return f'{forward_library[dex][0]}{fwd.upper()}{forward_library[dex][1]}'

def swap_structure1(dex, structure, start, end='today', bps=True):
    f_fly = structure.count('/') == 2
    f_crv = structure.count('/') == 1
    fly = structure.count('s') == 3
    crv = structure.count('s') == 2
    out = max(2 - sum([i.isalpha() for i in structure]), 0)

    if f_fly or f_crv:
        legs = [forward_ticker1(dex, i) for i in structure.split('/')]
    elif fly or crv:
        legs = [spot_ticker1(dex, i) for i in [i + 'Y' for i in structure.split('s') if i.isdigit()]]
    else:
        legs = forward_ticker1(dex, ('0Y' * out) + structure)

    df = blp.bdh(legs, flds='px_last', start_date=start, end_date=end).bfill() * (100 if bps else 1)
    
    if df.empty or df.shape[1] < 1:
        return None
    
    s = pd.DataFrame({})
    if f_fly or fly:
        x = (2 * df.iloc[:, 1]) - (df.iloc[:, 0] + df.iloc[:, 2])
    elif f_crv or crv:
        x = df.iloc[:, 1] - df.iloc[:, 0]
    else:
        x = df.iloc[:, 0]

    s[f'{dex.upper()} {structure}'] = x
    return s

# Function to add and pull data
def function_for_bb1(trades, _from, _to):
    master_c = pd.DataFrame({})
    for j in trades:
        h = swap_structure1(dex=j[0].lower(), structure=j[1].lower(), start=_from, end=_to)
        if h is None:
            continue
        if master_c.shape == (0, 0):
            master_c = h
        else:
            master_c = master_c.join(h, how='outer')
    return master_c

# Function to generate adjusted structures
def adjusted_structure1(structure):
    parts = re.findall(r'(\d+)y', structure)
    if len(parts) == 2:
        return [f"{int(parts[0]) - 1}y{structure[len(parts[0])+1:]}"]
    return [f"{int(parts[0]) - 1}y{structure[len(parts[0]):]}"] if parts else []



In [12]:
curr = ['eur', 'usd', 'nok', 'sek', 'jpy', 'cad', 'aud', 'gbp', 'chf', 'nzd']
tens = ['1y1y', '2y2y', '3y3y', '5y5y', '3y2y', '2y1y', '5y2y', '7y3y', '10y10y', '15y15y', '20y10y']

In [13]:
len1 = len(curr)-1
len2 = len(tens)

total = len1*len2

total

99

In [14]:
gen_from_1 = dt.strptime("01/01/2019", "%d/%m/%Y")
gen_from_2 = dt(dt.today().year - 4, dt.today().month, dt.today().day)
gen_to_1 = dt.today()
trades2 = []

for i in tens:
    for j in curr:
        trades2.append(tuple([j,i]))




In [15]:

master = function_for_bb1(trades2, gen_from_1, gen_to_1)

In [16]:
master

,EUR 1y1y,USD 1y1y,NOK 1y1y,SEK 1y1y,JPY 1y1y,AUD 1y1y,GBP 1y1y,CHF 1y1y,NZD 1y1y,EUR 2y2y,...,NZD 15y15y,EUR 20y10y,USD 20y10y,NOK 20y10y,SEK 20y10y,JPY 20y10y,AUD 20y10y,GBP 20y10y,CHF 20y10y,NZD 20y10y
2019-01-01,NaN,NaN,NaN,19.17,-5.38,199.26,95.76,-58.79,196.879,NaN,...,263.434,NaN,NaN,NaN,174.93,101.89,269.52,130.09,90.71,209.129
2019-01-02,NaN,NaN,NaN,16.72,-5.38,192.34,89.66,-61.12,191.549,NaN,...,259.992,NaN,NaN,NaN,168.08,101.89,259.23,128.59,87.74,205.429
2019-01-03,NaN,NaN,NaN,13.78,-5.62,189.75,87.02,-61.93,195.423,NaN,...,257.557,NaN,NaN,NaN,167.00,101.89,258.30,129.04,86.91,205.283
2019-01-04,NaN,NaN,NaN,16.14,-6.30,204.16,92.00,-62.04,197.552,NaN,...,256.643,NaN,NaN,NaN,170.50,90.94,272.95,133.03,89.23,206.056
2019-01-07,NaN,NaN,NaN,16.27,-6.30,200.85,91.36,-61.22,197.935,NaN,...,261.735,NaN,NaN,NaN,171.69,96.59,271.67,133.67,91.30,208.740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-27,210.502,313.180,336.293,193.36,54.13,352.52,371.62,59.46,327.653,212.838,...,462.558,191.641,284.030,242.354,164.18,205.25,361.87,368.83,59.05,461.547
2024-08-28,209.226,314.460,331.299,191.26,54.47,355.70,373.12,56.85,328.057,211.321,...,465.186,189.212,284.985,239.208,162.30,203.61,365.11,369.03,55.94,462.853
2024-08-29,206.584,316.281,327.629,189.75,53.43,358.72,371.96,55.40,340.972,211.953,...,469.888,192.603,285.863,240.089,167.89,211.76,368.12,370.35,57.53,468.362
2024-08-30,210.576,318.692,327.102,191.78,55.21,360.66,372.84,56.18,340.434,215.313,...,470.756,194.755,289.958,242.520,167.87,213.08,371.59,369.99,56.55,468.650


In [17]:
master.to_csv('trades.csv')

In [18]:
def generate_combinations():
    combinations = []
    # Iterate over the range of numbers from 0 to 98 with a step of 9
    for start in range(0, 99, 9):
        # Define the end of the current group
        end = min(start + 9, 99)
        # Generate the combinations within the current group
        for i in range(start, end):
            for j in range(i + 1, end):
                combinations.append([i, j])
    return combinations

# Usage
combinations = generate_combinations()

In [19]:
combinations

[[0, 1],
 [0, 2],
 [0, 3],
 [0, 4],
 [0, 5],
 [0, 6],
 [0, 7],
 [0, 8],
 [1, 2],
 [1, 3],
 [1, 4],
 [1, 5],
 [1, 6],
 [1, 7],
 [1, 8],
 [2, 3],
 [2, 4],
 [2, 5],
 [2, 6],
 [2, 7],
 [2, 8],
 [3, 4],
 [3, 5],
 [3, 6],
 [3, 7],
 [3, 8],
 [4, 5],
 [4, 6],
 [4, 7],
 [4, 8],
 [5, 6],
 [5, 7],
 [5, 8],
 [6, 7],
 [6, 8],
 [7, 8],
 [9, 10],
 [9, 11],
 [9, 12],
 [9, 13],
 [9, 14],
 [9, 15],
 [9, 16],
 [9, 17],
 [10, 11],
 [10, 12],
 [10, 13],
 [10, 14],
 [10, 15],
 [10, 16],
 [10, 17],
 [11, 12],
 [11, 13],
 [11, 14],
 [11, 15],
 [11, 16],
 [11, 17],
 [12, 13],
 [12, 14],
 [12, 15],
 [12, 16],
 [12, 17],
 [13, 14],
 [13, 15],
 [13, 16],
 [13, 17],
 [14, 15],
 [14, 16],
 [14, 17],
 [15, 16],
 [15, 17],
 [16, 17],
 [18, 19],
 [18, 20],
 [18, 21],
 [18, 22],
 [18, 23],
 [18, 24],
 [18, 25],
 [18, 26],
 [19, 20],
 [19, 21],
 [19, 22],
 [19, 23],
 [19, 24],
 [19, 25],
 [19, 26],
 [20, 21],
 [20, 22],
 [20, 23],
 [20, 24],
 [20, 25],
 [20, 26],
 [21, 22],
 [21, 23],
 [21, 24],
 [21, 25],
 [21, 26],
 [

In [20]:
a = pd.read_csv('W1.csv')
a

,Unnamed: 0,Name,Blue,Spread,Flag,End_Spread,End_Exp,Take_Profit,Total_Num,Win_Ratio,Avg_Time,Max_Drawdown,Cum_PNL,Average_PNL,Max_PNL,Fourier,ttof
